In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

data_path = Path("../datasets/raw_UKReg/")

In [ ]:
raw_regs = pd.read_csv(data_path / "liver_registrations.csv")

raw_regs

In [ ]:
raw_trans = pd.read_csv(data_path / "liver_transplant.csv")

raw_trans

In [ ]:
#full_df = pd.merge(raw_regs, raw_trans, left_on = 'a_registration_id', right_on = 'REGID')
raw_regs.loc[:, 'RECIPID'] = raw_regs['a_recip_id']
full_df = raw_regs.merge(raw_trans, how='left', on='RECIPID')

full_df.loc[:, "SERUM_POTASSIUM"] = full_df["SERUM_POTASSIUM"].replace([99.9], np.nan)
full_df.loc[:, "AFP_LEVEL"] = full_df["AFP_LEVEL"].replace([99999, 88888], np.nan)
full_df.loc[full_df['outcome'].isin(['A', 'T']), 'rwtime'] = np.nan
full_df.loc[:, 'PSURV'] = full_df['PSURV'].replace(np.nan, 0)
full_df.loc[:, 'rwtime'] = full_df['rwtime'].replace(np.nan, 0)


full_df = full_df[full_df['INR'].notna()]
full_df = full_df[full_df['outcome'].notna()]
full_df = full_df[full_df['SEX'].notna()]
full_df = full_df[full_df['SEX'] != 8]
full_df = full_df[full_df['SERUM_BILIRUBIN'].notna()]
full_df = full_df[full_df['INR'].notna()]
full_df = full_df[full_df['SERUM_CREATININE'].notna()]
full_df = full_df[full_df['SERUM_SODIUM'].notna()]


full_df.loc[:, 'Y'] = full_df['rwtime'] + full_df['PSURV']
full_df = full_df[full_df['Y'] > 0]

full_df["CENS"] =  (full_df.outcome != 'T').astype(int)
full_df["CENS"].value_counts()

In [ ]:
with_transplant = full_df[full_df["CENS"] == 0].copy()
no_transplant = full_df[full_df["CENS"] == 1].copy()

In [ ]:
from organsync.models.transplantbenefit import TBS

In [ ]:
TBS().predict(with_transplant.head(10))